In [1]:
import codemaster as cm
import numpy as np
import random
import gensim
from nltk.corpus import words
from functools import reduce
import heapq


D:\anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [18]:
model_wiki.save_word2vec_format(fname="wikipedia-vectors", binary=True)

In [14]:
wrds = np.genfromtxt('wordlist.csv', delimiter=',', dtype=str).tolist()

# randomly select 25 cards, then randomly select a side for each
board = [x[random.random() > 0.5].lower() for x in random.sample(wrds, 25)]

# set up game by assigning each card (agent) a role
player1 = board[:9]
player2 = board[9:17]
neutral = board[17:24]
assassin = board[24]


In [19]:

# load model pre-trained on Google News corpus (downloaded from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit)
model_gnews = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
model_wiki = gensim.models.KeyedVectors.load_word2vec_format("wikipedia-vectors.bin", binary=True)
# choose candidate clues from nltk's words corpus, which itself is drawn from the UNIX words file
dct = words.words('en')


AttributeError: module 'gensim.models' has no attribute 'KeyedVecotrs'

In [4]:
# returns potential clues which are closer to all members of targets than all members of avoids, sorted in order of relevance to targets
def clue (model, targets, avoids, similarity_f, score_f, n=1):
    topclues = []
    for word in dct:
        if word not in model.vocab:
            continue
        if avoids:
            target_distances = [similarity_f(model, word, target) for target in targets]
            avoid_distances = [similarity_f(model, word, avoid) for avoid in avoids]
            if max(avoid_distances) > min(target_distances):
                continue
        score = score_f([similarity_f(model, word, target) for target in targets])
        if reduce((lambda x, y: y not in word and word not in y and x), targets, True):
            if len(topclues) < n:
                heapq.heappush(topclues, (score, word))
            elif score > topclues[0][0]:
                heapq.heapreplace(topclues, (score, word))
    return sorted(topclues, key=lambda x: x[0], reverse=True)

In [5]:
def model_sim(model, word, target):
    return model.similarity(word, target)


In [15]:
candidates = clue(model_gnews, board[:2], [], model_sim, sum, n=5)
candidates_1 = clue(model_gnews, board[:2], [], model_sim, min, n=5)
print ("SUM")
print ('Top clues for pair', board[:2], ':', candidates)
print ("MIN")
print ('Top clues for pair', board[:2], ':', candidates_1)


D:\anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


SUM
Top clues for pair ['agent', 'vacuum'] : [(0.6577949821949005, 'cyclopentane'), (0.6358997821807861, 'void'), (0.5346177965402603, 'neutralizer'), (0.5331879258155823, 'agency'), (0.5154851078987122, 'anticorrosive')]
MIN
Top clues for pair ['agent', 'vacuum'] : [(0.32664225, 'cyclopentane'), (0.24732275, 'neutralizer'), (0.22673756, 'fixer'), (0.22448832, 'botulin'), (0.22108072, 'embolic')]


In [21]:
candidates = clue(model_wiki, board[:2], [], model_sim, sum, n=5)
candidates_1 = clue(model_wiki, board[:2], [], model_sim, min, n=5)
print ("SUM")
print ('Top clues for pair', board[:2], ':', candidates)
print ("MIN")
print ('Top clues for pair', board[:2], ':', candidates_1)


D:\anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


SUM
Top clues for pair ['agent', 'vacuum'] : [(0.6149706989526749, 'refrigerant'), (0.6125208139419556, 'device'), (0.6112737357616425, 'solvent'), (0.6090558767318726, 'cryogen'), (0.5995049998164177, 'fluid')]
MIN
Top clues for pair ['agent', 'vacuum'] : [(0.25143254, 'hemostatic'), (0.2470825, 'simulant'), (0.24146003, 'cryogen'), (0.240347, 'operator'), (0.23518406, 'welder')]


In [37]:
clues = []

for i in range(len(player2)):
    print (clues)
    for j in range(i+1,len(player2)):
        targets = []
        targets.append(player2[i])
        targets.append(player2[j])
        candidates = clue(model_gnews, targets, player1, model_sim, min, n=5)
        for candidate in candidates:
            if len(clues) < 15:
                heapq.heappush(clues, (candidate[0], candidate[1], targets[0], targets[1]))
            elif candidate[0] > clues[0][0]:
                heapq.heapreplace(clues, (candidate[0], candidate[1], targets[0], targets[1]))

[]


D:\anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(0.26550764, 'palm', 'maple', 'hollywood'), (0.2763111, 'sugar', 'maple', 'iron'), (0.26584077, 'chokecherry', 'maple', 'himalayas'), (0.28439158, 'silver', 'maple', 'iron'), (0.2793153, 'conifer', 'maple', 'himalayas'), (0.26961198, 'laurel', 'maple', 'hollywood'), (0.27526456, 'gold', 'maple', 'iron'), (0.4546165, 'cherry', 'maple', 'honey'), (0.38182122, 'pecan', 'maple', 'honey'), (0.4285342, 'plum', 'maple', 'honey'), (0.28867325, 'magnolia', 'maple', 'hollywood'), (0.44513336, 'clover', 'maple', 'honey'), (0.28495523, 'deodara', 'maple', 'himalayas'), (0.37311083, 'cranberry', 'maple', 'honey'), (0.28976148, 'crowberry', 'maple', 'himalayas')]
[(0.28976148, 'crowberry', 'maple', 'himalayas'), (0.29373997, 'tequila', 'honey', 'hollywood'), (0.294832, 'twirl', 'honey', 'drill'), (0.29974496, 'hillbilly', 'honey', 'hollywood'), (0.300376, 'peddler', 'honey', 'soldier'), (0.2971006, 'dad', 'honey', 'soldier'), (0.31069815, 'dixie', 'honey', 'hollywood'), (0.4546165, 'cherry', 'maple

In [38]:
clues_sorted = sorted(clues, key=lambda x: x[0], reverse=False)
for c in clues_sorted:
    print (c)
print (player2)
print (player1)

(0.3028673, 'sapper', 'drill', 'soldier')
(0.3031981, 'ferrocerium', 'drill', 'iron')
(0.30569524, 'carabineer', 'drill', 'soldier')
(0.30729616, 'cored', 'drill', 'iron')
(0.3076811, 'gigolo', 'honey', 'hollywood')
(0.30841583, 'ballyhoo', 'honey', 'hollywood')
(0.31069815, 'dixie', 'honey', 'hollywood')
(0.32354492, 'leatherneck', 'drill', 'soldier')
(0.33931053, 'platoon', 'drill', 'soldier')
(0.34924844, 'girl', 'honey', 'soldier')
(0.37311083, 'cranberry', 'maple', 'honey')
(0.38182122, 'pecan', 'maple', 'honey')
(0.4285342, 'plum', 'maple', 'honey')
(0.44513336, 'clover', 'maple', 'honey')
(0.4546165, 'cherry', 'maple', 'honey')
['maple', 'honey', 'drill', 'hollywood', 'soldier', 'iron', 'row', 'himalayas']
['rome', 'server', 'satellite', 'ice', 'foot', 'tooth', 'staff', 'ninja', 'brush']


In [37]:
clues_wiki = []

for i in range(len(player2)):
    print (clues_wiki)
    for j in range(i+1,len(player2)):
        targets = []
        targets.append(player2[i])
        targets.append(player2[j])
        candidates = clue(model_wiki, targets, player1, model_sim, min, n=5)
        for candidate in candidates:
            if len(clues_wiki) < 15:
                heapq.heappush(clues_wiki, (candidate[0], candidate[1], targets[0], targets[1]))
            elif candidate[0] > clues_wiki[0][0]:
                heapq.heapreplace(clues_wiki, (candidate[0], candidate[1], targets[0], targets[1]))

[]


D:\anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(0.26550764, 'palm', 'maple', 'hollywood'), (0.2763111, 'sugar', 'maple', 'iron'), (0.26584077, 'chokecherry', 'maple', 'himalayas'), (0.28439158, 'silver', 'maple', 'iron'), (0.2793153, 'conifer', 'maple', 'himalayas'), (0.26961198, 'laurel', 'maple', 'hollywood'), (0.27526456, 'gold', 'maple', 'iron'), (0.4546165, 'cherry', 'maple', 'honey'), (0.38182122, 'pecan', 'maple', 'honey'), (0.4285342, 'plum', 'maple', 'honey'), (0.28867325, 'magnolia', 'maple', 'hollywood'), (0.44513336, 'clover', 'maple', 'honey'), (0.28495523, 'deodara', 'maple', 'himalayas'), (0.37311083, 'cranberry', 'maple', 'honey'), (0.28976148, 'crowberry', 'maple', 'himalayas')]
[(0.28976148, 'crowberry', 'maple', 'himalayas'), (0.29373997, 'tequila', 'honey', 'hollywood'), (0.294832, 'twirl', 'honey', 'drill'), (0.29974496, 'hillbilly', 'honey', 'hollywood'), (0.300376, 'peddler', 'honey', 'soldier'), (0.2971006, 'dad', 'honey', 'soldier'), (0.31069815, 'dixie', 'honey', 'hollywood'), (0.4546165, 'cherry', 'maple

In [ ]:
clues_wiki_sorted = sorted(clues_wiki, key=lambda x: x[0], reverse=False)
for c in clues_wiki_sorted:
    print (c)
print (player2)
print (player1)